In [ ]:
%load_ext autoreload
%autoreload 2
import sys, os
sys.path.append('../')

In [ ]:
# specify parameters for interactive run
# this cell will be replaced during pipeline run with the parameters from notebook function call
train_params={'df_X_train': '/tmp/step/house_price-lite/exp-pipeline/run-24-12-27-053904/df_X_train',
 'df_Y_train': '/tmp/step/house_price-lite/exp-pipeline/run-24-12-27-053904/df_Y_train',
 'df_X_eval': '/tmp/step/house_price-lite/exp-pipeline/run-24-12-27-053904/df_X_eval',
 'df_Y_eval': '/tmp/step/house_price-lite/exp-pipeline/run-24-12-27-053904/df_Y_eval',
 'df_X_test': '/tmp/step/house_price-lite/exp-pipeline/run-24-12-27-053904/df_X_test',
 'df_Y_test': '/tmp/step/house_price-lite/exp-pipeline/run-24-12-27-053904/df_Y_test'}

In [ ]:
# run spark
from sinara.spark import SinaraSpark

spark = SinaraSpark.run_session(0)
SinaraSpark.ui_url()

In [ ]:
# read inputs 
df_X_train = spark.read.parquet(train_params["df_X_train"]).pandas_api()
df_Y_train = spark.read.parquet(train_params["df_Y_train"]).pandas_api()
df_X_eval = spark.read.parquet(train_params["df_X_eval"]).pandas_api()
df_Y_eval = spark.read.parquet(train_params["df_Y_eval"]).pandas_api()
df_X_test = spark.read.parquet(train_params["df_X_test"]).pandas_api()
df_Y_test = spark.read.parquet(train_params["df_Y_test"]).pandas_api()

In [ ]:
# Train the model predicting median house price (MEDV)
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import root_mean_squared_error

reg = GradientBoostingRegressor(random_state=239)
reg.fit(df_X_train.to_numpy(), df_Y_train.to_numpy())

In [ ]:
# check trained model quality on eval dataset using RMSE
rmse = root_mean_squared_error(df_Y_eval.to_numpy(), reg.predict(df_X_eval.to_numpy()))
print("The root mean squared error (RMSE) on eval set: {:.4f}".format(rmse))

In [ ]:
# create and save a bentoservice
from model_service import ModelService
from sinara.bentoml import save_bentoservice
from sinara.substep import get_tmp_work_path

test_data = {}
test_data['X'] = df_X_test.to_dict(orient='records')
test_data['Y'] = df_Y_test.to_dict(orient='records')

model = ModelService()
model.pack('model', reg)
model.pack('test_data', test_data)

output_bento_service_path = f"{get_tmp_work_path()}/bento_service"

save_bentoservice(model, path=output_bento_service_path, service_version='1')

In [ ]:
# stop spark
SinaraSpark.stop_session()

In [ ]:
# results used by the model_eval and model_test steps
output_bento_service_path